# What makes a good athelete? 

## Data cleaning, wrangling and merging

Data analysis of Olympic data from the Summer games. This file covers the data wrangling, cleaning and merging of the Olympic_Bio_Athlete.csv and Olympic_Athlete_Event_Results.csv data files to enable analysis of the peronal attributes that lead to successful atheltes.

## Background
The modern olympics see athletes competing in a wide range of sports. Visually there are numerous differences between athletes in each sport such as slight jumpers and powerful weightlifters but what makes Olympic medalists? This project will aim to investigate various factors that contributed to medal winners such as:
* height
* weight
* BMI
* age
* home Olympics
* population of country
* GDP of country
 
 This work will focus on 5 different sports:
* eventing
* 100 m
* marathon
* shot put
* high jump

### Import modules

In [24]:
import pandas as pd
import numpy as np

### Initial inspection of Olympic_Athlete_Bio.csv data

In [25]:
athlete_bio = pd.read_csv('Olympic_Athlete_Bio.csv')
print(athlete_bio.head())
print(athlete_bio.info())

   athlete_id                name     sex              born  height weight  \
0       65649       Ivanka Bonova  Female      4 April 1949   166.0     55   
1      112510   Nataliya Uryadova  Female     15 March 1977   184.0     70   
2      114973  Essa Ismail Rashed    Male  14 December 1986   165.0     55   
3       30359         Péter Boros    Male   12 January 1908     NaN    NaN   
4       50557      Rudolf Piowatý    Male     28 April 1900     NaN    NaN   

               country country_noc  \
0             Bulgaria         BUL   
1   Russian Federation         RUS   
2                Qatar         QAT   
3              Hungary         HUN   
4       Czechoslovakia         TCH   

                                         description  \
0                 Personal Best: 400 – 53.54 (1980).   
1                                                NaN   
2            Personal Best: 10000 – 27:20.97 (2006).   
3  Between 1927 and 1938, Péter Boros competed as...   
4  Rudolf Piowaty join

#### Notes
* There is a lot of data missing from the description and special_notes comments however, these are irrelvant to the investigations. 
* 2.6 % of entries are missing from the born column. The born column contains numerous types of answer.
* 32.6 % of entries are missing from the height and weight columns
* Weight has the wrong data type
* Is there a connection between entries with DOBs, weights and heights missing?

As this data will be looked at in connection with the athlete results, the two dataframes will be merged prior to data cleaning and wrangling.

### Initial inspection of Olympic_Athlete_Event_Results.csv data

In [26]:
athlete_results = pd.read_csv('Olympic_Athlete_Event_Results.csv')
print(athlete_results.head())
print(athlete_results.info())

                edition  edition_id country_noc      sport            event  \
0  1908 Summer Olympics           5         ANZ  Athletics  100 metres, Men   
1  1908 Summer Olympics           5         ANZ  Athletics  400 metres, Men   
2  1908 Summer Olympics           5         ANZ  Athletics  800 metres, Men   
3  1908 Summer Olympics           5         ANZ  Athletics  800 metres, Men   
4  1908 Summer Olympics           5         ANZ  Athletics  800 metres, Men   

   result_id          athlete  athlete_id        pos medal  isTeamSport  
0      56265  Ernest Hutcheon       64710        DNS   NaN        False  
1      56313     Henry Murray       64756        DNS   NaN        False  
2      56338    Harvey Sutton       64808  3 h8 r1/2   NaN        False  
3      56338      Guy Haskins      922519        DNS   NaN        False  
4      56338     Joseph Lynch       64735        DNS   NaN        False  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316834 entries, 0 to 316833
Dat

#### Notes
* All of the columns appear to have the correct data type
* The only column missing data is the medal column which is to be expected as the majority of atheltes taking part will not earn a medal.

### Combination of athlete_bio and athlete_results dataframes, selection of summer olympics and data cleaning and wrangling
The combined data has been filtered to just the Summer Olympics and team sports have also been filtered out as these rely on the combined attributes of the team members. 

In [27]:
combined_df = pd.merge(athlete_bio, athlete_results)
df_split = combined_df.edition.str.split(' ')
combined_df['Year'] = df_split.str.get(0)
combined_df['Year'] = combined_df['Year'].astype('int64')
combined_df['Olympics'] = df_split.str.get(1)

summer_olympics = combined_df[(combined_df.Olympics == 'Summer') & (combined_df.isTeamSport == False)].reset_index()

summer_olympics['born'] = pd.to_datetime(summer_olympics['born'], errors="coerce").dt.year

print(summer_olympics.head())
print(summer_olympics.info())

   index  athlete_id                name   sex    born  height weight  \
0      3      114973  Essa Ismail Rashed  Male  1986.0   165.0     55   
1      4       30359         Péter Boros  Male  1908.0     NaN    NaN   
2      6       30359         Péter Boros  Male  1908.0     NaN    NaN   
3      7       30359         Péter Boros  Male  1908.0     NaN    NaN   
4      8       30359         Péter Boros  Male  1908.0     NaN    NaN   

    country country_noc                                        description  \
0     Qatar         QAT            Personal Best: 10000 – 27:20.97 (2006).   
1   Hungary         HUN  Between 1927 and 1938, Péter Boros competed as...   
2   Hungary         HUN  Between 1927 and 1938, Péter Boros competed as...   
3   Hungary         HUN  Between 1927 and 1938, Péter Boros competed as...   
4   Hungary         HUN  Between 1927 and 1938, Péter Boros competed as...   

   ... edition_id                sport                       event result_id  \
0  ...      

#### Filtering the data has reduced the number of missing entries to:
* Born 0.5%
* Weight 11.4% 
* Height 11.4%

Do entries with missing data have any connection to each other?

In [28]:
summer_olympics_missing = combined_df[(combined_df.Olympics == 'Summer') & (combined_df.isTeamSport == False)].reset_index()
summer_olympics_missing['born'] = summer_olympics_missing['born'].fillna(value=3000)
summer_olympics_missing['height'] = summer_olympics_missing['height'].fillna(value=3000)

summer_olympics_missing_dob = summer_olympics_missing[summer_olympics_missing.born == 3000].reset_index()
print('DOB - Olympics:', summer_olympics_missing_dob.edition.value_counts())
print('DOB - Sport: ', summer_olympics_missing_dob.sport.value_counts())
print('DOB - Medal: ', summer_olympics_missing_dob.medal.value_counts())

summer_olympics_missing_height = summer_olympics_missing[summer_olympics_missing.height == 3000].reset_index()
print('Height - Olympics:', summer_olympics_missing_height.edition.value_counts())
print('Height - Sport: ', summer_olympics_missing_height.sport.value_counts())
print('Height - Medal: ', summer_olympics_missing_height.medal.value_counts())

DOB - Olympics: edition
1900 Summer Olympics    785
1904 Summer Olympics    491
1908 Summer Olympics    239
1928 Summer Olympics    159
1948 Summer Olympics    157
1896 Summer Olympics    154
1924 Summer Olympics    143
1920 Summer Olympics    132
1984 Summer Olympics    100
1980 Summer Olympics     73
1932 Summer Olympics     56
2008 Summer Olympics     45
1912 Summer Olympics     42
1952 Summer Olympics     41
1936 Summer Olympics     37
1972 Summer Olympics     31
1988 Summer Olympics     22
1992 Summer Olympics     17
1976 Summer Olympics     15
1968 Summer Olympics      9
1960 Summer Olympics      7
1956 Summer Olympics      5
1964 Summer Olympics      3
1996 Summer Olympics      3
2000 Summer Olympics      1
Name: count, dtype: int64
DOB - Sport:  sport
Athletics              789
Swimming               335
Artistic Gymnastics    214
Fencing                182
Boxing                 180
Cycling Track          162
Art Competitions       142
Shooting               103
Tennis        

The majority of the missing data seems to be from the earlier years. The data will therefore be filtered to include only the last 5 Olympic Games and the above analyses re-ran. This is also more representative of the current day as the dimensions of the body have changed over time. On doing this it was discovered that a lot of data was missing from 2020. This is likely due to the impact of COVID and therefore this year will also be ommited.

In [29]:
summer_olympics_trimmed = summer_olympics[(summer_olympics.Year >= 2004) & (summer_olympics.Year < 2020)].reset_index()

print(summer_olympics_trimmed.info())
summer_olympics_trimmed['born'] = summer_olympics_trimmed['born'].fillna(value=3000)
summer_olympics_trimmed['height'] = summer_olympics_trimmed['height'].fillna(value=3000)

summer_olympics_trimmed_dob = summer_olympics_trimmed[summer_olympics_trimmed.born == 3000]
print('DOB - Olympics:', summer_olympics_trimmed_dob.edition.value_counts())
print('DOB - Sport: ', summer_olympics_trimmed_dob.sport.value_counts())
print('DOB - Medal: ', summer_olympics_trimmed_dob.medal.value_counts())

summer_olympics_trimmed_height = summer_olympics_trimmed[summer_olympics_trimmed.height == 3000]
print('Height - Olympics:', summer_olympics_trimmed_height.edition.value_counts())
print('Height - Sport: ', summer_olympics_trimmed_height.sport.value_counts())
print('Height - Medal: ', summer_olympics_trimmed_height.medal.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32684 entries, 0 to 32683
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   level_0        32684 non-null  int64  
 1   index          32684 non-null  int64  
 2   athlete_id     32684 non-null  int64  
 3   name           32684 non-null  object 
 4   sex            32684 non-null  object 
 5   born           32630 non-null  float64
 6   height         32045 non-null  float64
 7   weight         32045 non-null  object 
 8   country        32684 non-null  object 
 9   country_noc    32684 non-null  object 
 10  description    7158 non-null   object 
 11  special_notes  19855 non-null  object 
 12  edition        32684 non-null  object 
 13  edition_id     32684 non-null  int64  
 14  sport          32684 non-null  object 
 15  event          32684 non-null  object 
 16  result_id      32684 non-null  int64  
 17  athlete        32684 non-null  object 
 18  pos   

* This data cleaning has resulted in only 0.17% of missing born data and 1.96% of missing height/weight data. 
* The missing data for born is mainly for Wushu which was a sport only at the Beijing Olympics in 2008
* As the data is missing at random then the rows containing missing infomation can be deleted.

In [30]:
summer_olympics_cleaned = summer_olympics[(summer_olympics.Year >= 2004) & (summer_olympics.Year < 2020)].reset_index()
summer_olympics_cleaned = summer_olympics_cleaned.dropna(subset=['born', 'height', 'weight'])
print(summer_olympics_cleaned.head())
print(summer_olympics_cleaned.info())
print(summer_olympics_cleaned.weight.unique())

   level_0  index  athlete_id                  name     sex    born  height  \
0        0      3      114973    Essa Ismail Rashed    Male  1986.0   165.0   
1        7     13      133041      Vincent Riendeau    Male  1996.0   178.0   
2       20     47      115157      Mariya Yakovenko  Female  1982.0   174.0   
3       26     66      136267  Shadrack Kipchirchir    Male  1989.0   173.0   
4       27     70      112346     Paul Etia Ndoumbè    Male  1984.0   182.0   

  weight              country country_noc  ... edition_id      sport  \
0     55                Qatar         QAT  ...         53  Athletics   
1     68               Canada         CAN  ...         59     Diving   
2     81   Russian Federation         RUS  ...         53  Athletics   
3     54        United States         USA  ...         59  Athletics   
4     78             Cameroon         CMR  ...         53     Rowing   

                  event  result_id               athlete      pos  medal  \
0    10,000 metr

It was noted that some athletes gave their weight as a range or multiple values. In order to make this data suitable for analysis, the average of these results will be collated. On inspecting the results through Pandas and excel an unusal weight of 77,5 was found for Nick Buckfield. A google search found that this is an error online suggesting that he is 775 kg. As a result this result will be deleted. Other google searches found \ax0 to be an error but the preceeding weight is correct. Therefore this will also be corrected in the results.

In [31]:
summer_olympics_cleaned = summer_olympics_cleaned[summer_olympics_cleaned.name != 'Nick Buckfield']
summer_olympics_cleaned['weight'] = summer_olympics_cleaned['weight'].replace('\xa0', '', regex=True)

summer_olympics_cleaned[['weight_low', 'weight_high']] = summer_olympics_cleaned['weight'].str.split('[-,]', expand=True)
summer_olympics_cleaned[['weight_low', 'weight_high']] = summer_olympics_cleaned[['weight_low', 'weight_high']].astype('float')
summer_olympics_cleaned = summer_olympics_cleaned.fillna(value={'weight_high':summer_olympics_cleaned.weight_low})


summer_olympics_cleaned['weight_average'] = (summer_olympics_cleaned['weight_low'] + summer_olympics_cleaned['weight_high']) * 0.5

### Calculation weight, BMI and age

In [32]:
summer_olympics_cleaned['BMI'] = summer_olympics_cleaned['weight_average'] / ((summer_olympics_cleaned['height']/100) ** 2) 
summer_olympics_cleaned['Age'] = summer_olympics_cleaned['Year'] - summer_olympics_cleaned['born']

print(summer_olympics_cleaned.head())
print(summer_olympics_cleaned.height.describe())
print(summer_olympics_cleaned.born.describe())


   level_0  index  athlete_id                  name     sex    born  height  \
0        0      3      114973    Essa Ismail Rashed    Male  1986.0   165.0   
1        7     13      133041      Vincent Riendeau    Male  1996.0   178.0   
2       20     47      115157      Mariya Yakovenko  Female  1982.0   174.0   
3       26     66      136267  Shadrack Kipchirchir    Male  1989.0   173.0   
4       27     70      112346     Paul Etia Ndoumbè    Male  1984.0   182.0   

  weight              country country_noc  ...      pos medal isTeamSport  \
0     55                Qatar         QAT  ...       20   NaN       False   
1     68               Canada         CAN  ...  14 r2/3   NaN       False   
2     81   Russian Federation         RUS  ...  46 r1/2   NaN       False   
3     54        United States         USA  ...       19   NaN       False   
4     78             Cameroon         CMR  ...       28   NaN       False   

   Year Olympics weight_low  weight_high weight_average       

In [33]:
summer_olympics_final  = summer_olympics_cleaned[summer_olympics_cleaned.sex == 'Male'] 


print(summer_olympics_final.head())


   level_0  index  athlete_id                  name   sex    born  height  \
0        0      3      114973    Essa Ismail Rashed  Male  1986.0   165.0   
1        7     13      133041      Vincent Riendeau  Male  1996.0   178.0   
3       26     66      136267  Shadrack Kipchirchir  Male  1989.0   173.0   
4       27     70      112346     Paul Etia Ndoumbè  Male  1984.0   182.0   
5       28     71      112346     Paul Etia Ndoumbè  Male  1984.0   182.0   

  weight         country country_noc  ...      pos medal isTeamSport  Year  \
0     55           Qatar         QAT  ...       20   NaN       False  2008   
1     68          Canada         CAN  ...  14 r2/3   NaN       False  2016   
3     54   United States         USA  ...       19   NaN       False  2016   
4     78        Cameroon         CMR  ...       28   NaN       False  2008   
5     78        Cameroon         CMR  ...       32   NaN       False  2012   

  Olympics weight_low  weight_high weight_average        BMI   Age  

#### Saving modified database as a new csv file

In [34]:
summer_olympics_final.to_csv(r'athlete_results_bio.csv')